In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import datetime as dt

In [ ]:
class Backtester:
    """
    A class to perform cross-sectional backtesting on CRSP-style long-format data.

    This framework is inspired by the portfolio sorting methodology used in academic
    finance papers like Ang et al. (2006). It sorts assets into quantiles
    based on a signal at each rebalancing period and calculates the performance
    of long-short portfolios.

    Attributes:
        data (pd.DataFrame): DataFrame in long format with columns for date, permno,
                             total_return, and the signal.
        signal_col (str): The name of the column in `data` that contains the signal values.
        rebalance_freq (str): Pandas frequency string for rebalancing.
        n_quantiles (int): The number of quantiles to sort the assets into.
        long_high_signal (bool): If True, the top quantile is the long portfolio;
                                 if False, the bottom quantile is the long portfolio.
        portfolio_returns (pd.DataFrame): Stores the daily returns of the portfolios.
        turnover (pd.Series): Stores the average annual turnover.
    """
    def __init__(self, data, signal_col, total_return_col='DlyRet', permno_col='PERMNO', date_col='DlyCalDt',
                 rebalance_freq='M', n_quantiles=5, long_high_signal=True):
        """
        Initializes the Backtester.

        Args:
            data (pd.DataFrame): Long-format DataFrame containing the backtest data.
            signal_col (str): The name of the signal column.
            total_return_col (str): The name of the total return column.
            permno_col (str): The name of the security identifier column.
            date_col (str): The name of the date column.
            rebalance_freq (str, optional): Rebalancing frequency. Defaults to 'M'.
            n_quantiles (int, optional): Number of portfolios to form. Defaults to 5.
            long_high_signal (bool, optional): Defines portfolio formation logic. Defaults to True.
        """
        # --- Validation ---
        required_cols = [date_col, permno_col, total_return_col, signal_col]
        if not all(col in data.columns for col in required_cols):
            raise ValueError(f"Input DataFrame is missing one or more required columns: {required_cols}")

        self.data = data.copy()
        # Ensure date column is datetime
        self.data[date_col] = pd.to_datetime(self.data[date_col])

        self.signal_col = signal_col
        self.return_col = total_return_col
        self.permno_col = permno_col
        self.date_col = date_col
        self.rebalance_freq = rebalance_freq
        self.n_quantiles = n_quantiles
        self.long_high_signal = long_high_signal
        
        self.portfolio_returns = None
        self.turnover = None

        print("Backtester initialized for CRSP-style data.")
        print(f"Number of securities: {self.data[self.permno_col].nunique()}")
        print(f"Date range: {self.data[self.date_col].min().date()} to {self.data[self.date_col].max().date()}")

    def _get_rebalance_dates(self):
        """Determines the dates on which to rebalance the portfolios."""
        all_dates = self.data[self.date_col].unique()
        # Resample to the desired frequency and get the last day of the period.
        rebalance_dates = pd.to_datetime(all_dates).to_series().resample(self.rebalance_freq).max().dropna()
        return rebalance_dates

    def run_backtest(self):
        """
        Runs the full portfolio sorting backtest.
        """
        print(f"\nRunning backtest with {self.n_quantiles} quantiles and '{self.rebalance_freq}' rebalancing...")
        print(f"Long portfolio: {'Highest' if self.long_high_signal else 'Lowest'} signal quantile.")

        rebalance_dates = self._get_rebalance_dates()
        all_daily_returns = []
        last_portfolio_holdings = set()

        # Loop through each rebalancing period
        for i in range(len(rebalance_dates) - 1):
            start_date = rebalance_dates.iloc[i]
            end_date = rebalance_dates.iloc[i+1]

            # Get the signal data on the rebalancing date
            rebalance_data = self.data[self.data[self.date_col] == start_date]
            current_signal = rebalance_data.set_index(self.permno_col)[self.signal_col].dropna()

            if current_signal.empty:
                continue

            # --- Portfolio Formation ---
            try:
                quantile_labels = pd.qcut(current_signal, self.n_quantiles, labels=False, duplicates='drop')
            except ValueError:
                print(f"Warning: Could not form {self.n_quantiles} quantiles on {start_date.date()}. Skipping period.")
                continue

            # Assign permnos to long and short portfolios based on the strategy logic
            if self.long_high_signal:
                long_permnos = set(quantile_labels[quantile_labels == self.n_quantiles - 1].index)
                short_permnos = set(quantile_labels[quantile_labels == 0].index)
            else:
                long_permnos = set(quantile_labels[quantile_labels == 0].index)
                short_permnos = set(quantile_labels[quantile_labels == self.n_quantiles - 1].index)

            if not long_permnos or not short_permnos:
                continue

            # --- Turnover Calculation ---
            current_portfolio_holdings = long_permnos.union(short_permnos)
            if i > 0:
                traded_assets = len(current_portfolio_holdings.symmetric_difference(last_portfolio_holdings))
                total_assets = len(current_portfolio_holdings) + len(last_portfolio_holdings)
                period_turnover = traded_assets / total_assets if total_assets > 0 else 0
            else:
                period_turnover = 1.0  # First period turnover is 100%
            last_portfolio_holdings = current_portfolio_holdings.copy()

            # --- Return Calculation for the Holding Period ---
            holding_period_data = self.data[(self.data[self.date_col] > start_date) & (self.data[self.date_col] <= end_date)]

            # Calculate daily equal-weighted returns for each portfolio
            long_returns_series = holding_period_data[holding_period_data[self.permno_col].isin(long_permnos)]
            daily_long_ret = long_returns_series.groupby(self.date_col)[self.return_col].mean()

            short_returns_series = holding_period_data[holding_period_data[self.permno_col].isin(short_permnos)]
            daily_short_ret = short_returns_series.groupby(self.date_col)[self.return_col].mean()

            # The long-short strategy return
            hedge_returns = daily_long_ret - daily_short_ret

            period_df = pd.DataFrame({'long': daily_long_ret, 'short': daily_short_ret, 'long_short': hedge_returns})
            period_df['turnover'] = period_turnover
            all_daily_returns.append(period_df)

        if not all_daily_returns:
            print("Backtest generated no returns. Check data and date ranges.")
            return

        self.portfolio_returns = pd.concat(all_daily_returns).fillna(0)
        self.turnover = self.portfolio_returns['turnover'].resample('Y').mean()

        print("Backtest complete.")

    def get_performance_summary(self, risk_free_rate=0.02):
        """
        Calculates and prints key performance metrics for the backtest.

        Args:
            risk_free_rate (float, optional): Annual risk-free rate for Sharpe ratio calculation. Defaults to 0.02.
        """
        if self.portfolio_returns is None or self.portfolio_returns.empty:
            print("Please run the backtest first or check if it generated returns.")
            return

        summary = pd.DataFrame()
        daily_rf = (1 + risk_free_rate)**(1/252) - 1

        for portfolio in ['long', 'short', 'long_short']:
            returns = self.portfolio_returns[portfolio]
            cum_returns = (1 + returns).cumprod()

            annual_return = returns.mean() * 252
            annual_vol = returns.std() * np.sqrt(252)
            sharpe_ratio = (returns.mean() - daily_rf) / returns.std() * np.sqrt(252) if returns.std() > 0 else 0.0

            running_max = np.maximum.accumulate(cum_returns)
            drawdown = (cum_returns - running_max) / running_max
            max_drawdown = drawdown.min()

            summary[portfolio] = {
                'Cumulative Return': f"{cum_returns.iloc[-1]:.2%}",
                'Annualized Return': f"{annual_return:.2%}",
                'Annualized Volatility': f"{annual_vol:.2%}",
                'Sharpe Ratio': f"{sharpe_ratio:.2f}",
                'Max Drawdown': f"{max_drawdown:.2%}"
            }

        print("\n--- Performance Summary ---")
        print(summary)
        print("\n--- Average Annual Turnover ---")
        print(self.turnover.apply('{:.2%}'.format))

    def run_factor_regression(self, factors):
        """
        Performs a factor regression (e.g., Fama-French) on the long-short portfolio.

        Args:
            factors (pd.DataFrame): DataFrame with a DatetimeIndex and factor returns (e.g., Mkt-RF, SMB, HML).
        """
        if self.portfolio_returns is None or self.portfolio_returns.empty:
            print("Please run the backtest first or check if it generated returns.")
            return

        # Align data
        common_data = pd.concat([self.portfolio_returns['long_short'], factors], axis=1).dropna()
        common_data.rename(columns={'long_short': 'portfolio_excess_return'}, inplace=True)

        y = common_data['portfolio_excess_return']
        X = common_data[factors.columns]
        X = sm.add_constant(X)

        model = sm.OLS(y, X).fit()

        print("\n--- Factor Regression (Long-Short Portfolio) ---")
        alpha = model.params.iloc[0] * 252
        print(f"Annualized Alpha: {alpha:.2%}")
        print(f"T-stat(Alpha): {model.tvalues.iloc[0]:.2f}")
        print("\nFactor Betas:")
        print(model.params.iloc[1:])
        print(f"\nR-squared: {model.rsquared:.3f}")



In [32]:
data = pd.read_csv('../data/Capstone006ReturnsShort.zip', compression='zip') 
data['DlyCalDt'] = pd.to_datetime(data['DlyCalDt'])
data.head()

,PERMNO,HdrCUSIP,CUSIP,SecurityNm,Ticker,PERMCO,YYYYMMDD,DlyCalDt,DlyPrc,DlyCap,...,DlyVol,DlyClose,DlyLow,DlyHigh,DlyBid,DlyAsk,DlyOpen,DlyNumTrd,DisFacPr,DisFacShr
0,10104,68389X10,68389X10,ORACLE CORP; COM NONE; CONS,ORCL,8045,20220630,2022-06-30,69.87,1.861984e+08,...,9888767.0,69.87,68.210,70.25,69.87,69.89,68.21,NaN,NaN,NaN
1,10104,68389X10,68389X10,ORACLE CORP; COM NONE; CONS,ORCL,8045,20220701,2022-07-01,70.87,1.888633e+08,...,7805555.0,70.87,69.810,70.98,70.86,70.87,69.85,NaN,NaN,NaN
2,10104,68389X10,68389X10,ORACLE CORP; COM NONE; CONS,ORCL,8045,20220705,2022-07-05,71.99,1.918480e+08,...,9043000.0,71.99,70.180,72.00,71.99,72.00,70.38,NaN,NaN,NaN
3,10104,68389X10,68389X10,ORACLE CORP; COM NONE; CONS,ORCL,8045,20220706,2022-07-06,71.66,1.909686e+08,...,6043679.0,71.66,71.310,72.74,71.67,71.68,72.32,NaN,NaN,NaN
4,10104,68389X10,68389X10,ORACLE CORP; COM NONE; CONS,ORCL,8045,20220707,2022-07-07,71.83,1.914216e+08,...,7814621.0,71.83,71.175,71.98,71.82,71.83,71.83,NaN,NaN,NaN


In [33]:
dataclean = data.pivot_table(
    index='DlyCalDt', 
    columns='PERMNO', 
    values=['DlyRet', 'DlyCap'],
    aggfunc='mean' # Use 'mean' to average the duplicates
)

In [37]:
dataclean['DlyRet'][dataclean['DlyRet'].isna().any(axis=1)]

PERMNO,10104,10107,10138,10145,10516,10696,11308,11403,11404,11600,...,92611,92614,92655,93002,93089,93096,93132,93246,93429,93436
DlyCalDt,,,,,,,,,,,,,,,,,,,,,
2022-06-30,0.009536,-0.013179,-0.016364,0.003522,0.004661,-0.021232,0.002869,-0.005897,0.004012,-0.018137,...,-0.013083,0.020230,-0.004033,-0.009380,0.010332,-0.006517,-0.010320,-0.018778,-0.006234,-0.017579
2022-07-01,0.014312,0.010707,0.014963,0.007479,-0.015335,0.040575,0.023367,-0.002599,0.022503,0.013154,...,0.011631,0.031324,0.007340,-0.016406,0.012768,0.005949,0.013963,0.032909,0.013164,0.012429
2022-07-05,0.015804,0.012597,0.015350,-0.017246,-0.052742,0.016526,-0.017086,0.013365,-0.046791,-0.002762,...,0.006828,-0.024637,-0.023502,-0.003223,0.000685,0.015019,0.028412,-0.003770,0.024067,0.025536
2022-07-06,-0.004584,0.012783,-0.002221,0.006508,-0.002211,-0.002656,0.002054,0.029016,0.004423,-0.013296,...,0.002892,0.017307,0.019892,0.013248,0.015392,0.012517,0.014746,0.002261,0.002469,-0.005721
2022-07-07,0.002372,0.008227,0.017976,0.003349,0.030186,-0.005434,-0.007885,0.026596,-0.008593,-0.004492,...,0.008353,0.006503,-0.001766,0.033008,-0.000505,0.006636,0.029230,0.060365,0.012401,0.055279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-24,0.039751,0.008457,0.011810,-0.000623,0.002083,0.012431,0.006739,0.011037,-0.009491,0.010336,...,0.022924,-0.012570,0.016429,0.039406,0.004929,-0.014267,0.012312,0.018348,-0.003694,-0.023546
2025-06-25,-0.021136,0.004407,-0.001577,-0.012511,-0.011903,-0.013321,-0.008261,0.002460,-0.021359,0.023018,...,-0.018116,-0.014981,-0.011779,0.003336,-0.018585,-0.004766,-0.006991,0.025298,0.004929,-0.037948
2025-06-26,0.009966,0.010523,0.012954,0.020695,0.016635,0.005870,-0.002298,0.020704,0.003672,0.000000,...,0.002230,-0.011639,0.001987,0.020858,0.003156,-0.001241,0.001640,0.020729,-0.018446,-0.005404


In [40]:
nan_counts = dataclean['DlyRet'].isnull().sum()
print("NaN counts per column:")
print(nan_counts)

NaN counts per column:
PERMNO
10104    0
10107    0
10138    0
10145    0
10516    0
        ..
93096    0
93132    0
93246    0
93429    0
93436    0
Length: 520, dtype: int64


In [42]:
nan_counts[nan_counts>0]

PERMNO
11786    577
12448    542
14763    130
15069    147
19515    332
19521    445
19522    714
21186    245
22179    172
23604    112
23696    150
24039    272
24040    262
25869    607
75241    288
79678    427
81696    373
90090    577
92121     28
92257    399
dtype: int64